In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

## Prophet Feature Selection, so Dropping All Lag and Rolling

In [2]:
covid_df = pd.read_csv('penn_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})
covid_df = covid_df.drop(['new_cases_1 lag', 'new_cases_2_lag', 'new_cases_3_lag', 'new_cases_4_lag', 'new_cases_7_lag', 'new_cases_30_lag', 'new_cases_50_lag', 'new_cases_150_lag', 'new_cases_350_lag', 'new_deaths_1_lag', 'new_deaths_2_lag', 'new_deaths_3_lag', 'new_deaths_4_lag', 'new_deaths_7_lag', 'new_deaths_30_lag', 'new_deaths_50_lag'], axis=1)

train_size = int(len(covid_df)* 0.85)
covid_train = covid_df[0:train_size] # notice here the original test test is never used

In [3]:
train_size

671

In [4]:
X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
y_train = covid_train['y']

In [5]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_1 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_1[1].to_list())

['new_hospitalized_patients', 'new_cases_10_mean', 'new_cases_10_max', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_50_max', 'new_cases_10_std', 'new_cases_10_min', 'current_intensive_care_patients', 'new_deaths_10_mean', 'new_cases_50_min', 'minimum_temperature_celsius', 'Booster_Doses_18Plus', 'new_cases_100_std', 'new_deaths_10_std', 'Series_Complete_Yes', 'vaccination_rate_complete', 'new_cases_50_std', 'average_temperature_celsius', 'new_cases_50_mean', 'Administered_Dose1_Recip_65Plus', 'Administered_Dose1_Recip', 'relative_humidity', 'rainfall_mm', 'new_deaths_100_mean', 'new_cases_100_min', 'Administered_Dose1_Recip_5Plus', 'Series_Complete_5Plus', 'Series_Complete_18Plus', 'new_deaths_100_std', 'Administered_Dose1_Recip_12Plus']


In [6]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_2 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_2[1].to_list())

['new_cases_10_std', 'new_cases_10_mean', 'PEOPLE_DEATH_NEW_COUNT', 'new_hospitalized_patients', 'relative_humidity', 'minimum_temperature_celsius', 'current_intensive_care_patients', 'new_cases_50_mean', 'new_cases_10_max', 'rainfall_mm', 'maximum_temperature_celsius', 'new_cases_10_min', 'new_cases_50_std', 'average_temperature_celsius', 'new_deaths_10_mean', 'new_deaths_50_std', 'Administered_Dose1_Recip_5Plus', 'dew_point', 'new_deaths_10_std', 'new_deaths_50_mean', 'new_deaths_100_std', 'new_cases_100_std', 'new_cases_100_mean', 'Administered_Dose1_Recip', 'new_deaths_10_min', 'Series_Complete_5Plus', 'new_deaths_100_mean', 'Booster_Doses_65Plus', 'Booster_Doses', 'Series_Complete_65Plus']


In [7]:
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier

model_c = GradientBoostingRegressor().fit(X_train, y_train)
predictors_3 = pd.concat([pd.Series(model_c.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_3[1].to_list())

['new_cases_10_max', 'new_hospitalized_patients', 'new_cases_10_mean', 'new_cases_10_min', 'current_intensive_care_patients', 'new_cases_50_std', 'PEOPLE_DEATH_NEW_COUNT', 'Series_Complete_5Plus', 'Booster_Doses_18Plus', 'Series_Complete_Yes', 'Administered_Dose1_Recip_65Plus', 'new_cases_10_std', 'new_deaths_100_mean', 'new_deaths_50_std', 'new_deaths_10_std', 'Booster_Doses', 'vaccination_rate_complete', 'minimum_temperature_celsius', 'new_deaths_50_mean', 'new_deaths_10_mean', 'average_temperature_celsius', 'new_cases_100_mean', 'Booster_Doses_65Plus', 'dew_point', 'Series_Complete_12Plus', 'new_cases_100_max', 'new_deaths_100_std', 'new_cases_50_mean', 'relative_humidity', 'maximum_temperature_celsius']


In [8]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier

model_d = AdaBoostRegressor(random_state=1, base_estimator = DecisionTreeRegressor(random_state=1)).fit(X_train, y_train)
predictors_4 = pd.concat([pd.Series(model_d.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_4[1].to_list())

['new_cases_10_mean', 'new_cases_10_max', 'new_hospitalized_patients', 'current_intensive_care_patients', 'new_cases_50_std', 'rainfall_mm', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_std', 'new_cases_10_min', 'minimum_temperature_celsius', 'Series_Complete_65Plus', 'Booster_Doses', 'new_deaths_10_min', 'Series_Complete_Yes', 'average_temperature_celsius', 'vaccination_rate_complete', 'Booster_Doses_65Plus', 'new_deaths_10_mean', 'Booster_Doses_50Plus', 'new_deaths_100_mean', 'Series_Complete_5Plus', 'Series_Complete_12Plus', 'Series_Complete_18Plus', 'new_cases_100_mean', 'Administered_Dose1_Recip_12Plus', 'booster_rate', 'maximum_temperature_celsius', 'new_deaths_50_mean', 'relative_humidity', 'new_cases_100_std']


In [9]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier

model_e = RandomForestRegressor().fit(X_train, y_train)
predictors_5 = pd.concat([pd.Series(model_e.feature_importances_), pd.Series(X_train.columns)], axis=1).sort_values(by=0, ascending = False).head(30)
print(predictors_5[1].to_list())

['new_cases_10_max', 'new_cases_10_mean', 'new_hospitalized_patients', 'new_cases_10_min', 'current_intensive_care_patients', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_std', 'new_cases_50_std', 'Booster_Doses_50Plus', 'vaccination_rate_complete', 'booster_rate', 'minimum_temperature_celsius', 'Series_Complete_12Plus', 'Booster_Doses_65Plus', 'Booster_Doses', 'relative_humidity', 'new_deaths_10_mean', 'Booster_Doses_18Plus', 'new_deaths_10_min', 'maximum_temperature_celsius', 'Series_Complete_18Plus', 'rainfall_mm', 'Series_Complete_5Plus', 'new_cases_100_mean', 'new_cases_50_min', 'average_temperature_celsius', 'new_deaths_10_std', 'new_deaths_50_std', 'dew_point', 'new_cases_50_mean']


In [10]:
# using MARS summary for feature selection
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss') # note, terms in brackets are the hyperparameters 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
---------------------------------------------------------------------
Basis Function                                  Pruned  Coefficient  
---------------------------------------------------------------------
(Intercept)                                     No      805.655      
h(new_cases_10_mean-9356.9)                     No      2.50504      
h(9356.9-new_cases_10_mean)                     No      -1.32899     
h(Series_Complete_Yes-7.4857E+06)               No      0.193559     
h(7.4857E+06-Series_Complete_Yes)               No      0.0608377    
h(PEOPLE_DEATH_NEW_COUNT-156)                   Yes     None         
h(156-PEOPLE_DEATH_NEW_COUNT)                   No      -10.0026     
new_deaths_10_mean                              Yes     None         
h(new_cases_10_min-6578)                        No      -0.867931    
h(6578-new_cases_10_min)                        No      0.199383     
h(Administered_Dose1_Recip_18Plus-9.16048E+06)  No      -0.520513    
h(9.1604

In [11]:
def common_member(a, b, c, d, e):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
    set_4 = set(d)
    set_5 = set(e)
 
    if (set_1 & set_2 & set_3 & set_4 & set_5):
        print(set_1 & set_2 & set_3 & set_4 & set_5)


In [12]:
a = predictors_1[1].to_list()
b = predictors_2[1].to_list()
c = predictors_3[1].to_list()
d = predictors_4[1].to_list()
e = predictors_5[1].to_list()

In [13]:
full = common_member(a, b, c, d, e)
full

{'new_cases_50_std', 'PEOPLE_DEATH_NEW_COUNT', 'relative_humidity', 'average_temperature_celsius', 'new_cases_10_std', 'new_hospitalized_patients', 'Series_Complete_5Plus', 'new_deaths_10_mean', 'current_intensive_care_patients', 'new_cases_10_min', 'new_cases_10_mean', 'new_cases_10_max', 'minimum_temperature_celsius'}


## XGBoost Feature Selection, includes lag and rolling

In [ ]:
import os
os._exit(00)

In [22]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

In [23]:
covid_df = pd.read_csv('penn_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})

train_size = int(len(covid_df)* 0.85)
covid_train = covid_df[0:train_size]

X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
X_train = X_train.astype('float32')
y_train = covid_train['y']

In [24]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_7 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(40)
print(predictors_7[1].to_list())

['new_cases_7_lag', 'new_cases_10_max', 'new_hospitalized_patients', 'new_cases_1 lag', 'current_intensive_care_patients', 'new_cases_10_mean', 'new_cases_10_min', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_std', 'new_cases_4_lag', 'new_deaths_3_lag', 'new_deaths_4_lag', 'new_deaths_1_lag', 'new_cases_50_std', 'new_cases_50_max', 'new_cases_3_lag', 'new_cases_100_mean', 'new_cases_350_lag', 'Booster_Doses_65Plus', 'Completeness_pct', 'new_cases_50_min', 'Series_Complete_18Plus', 'Administered_Dose1_Recip', 'Booster_Doses', 'Administered_Dose1_Recip_18Plus', 'Booster_Doses_50Plus', 'new_deaths_10_std', 'new_cases_2_lag', 'relative_humidity', 'new_cases_30_lag', 'new_deaths_50_max', 'vaccination_rate_complete', 'minimum_temperature_celsius', 'new_deaths_7_lag', 'Series_Complete_5Plus', 'new_cases_100_std', 'Administered_Dose1_Recip_5Plus', 'new_deaths_10_mean', 'Series_Complete_12Plus', 'average_temperature_celsius']


In [25]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_8 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_8[1].to_list())

['new_cases_10_std', 'new_cases_10_mean', 'new_cases_7_lag', 'new_cases_1 lag', 'new_hospitalized_patients', 'new_cases_3_lag', 'new_deaths_3_lag', 'new_cases_4_lag', 'new_cases_30_lag', 'new_cases_2_lag', 'new_cases_10_min', 'new_deaths_4_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_max', 'new_cases_350_lag', 'relative_humidity', 'new_deaths_1_lag', 'new_cases_50_std', 'new_deaths_50_lag', 'minimum_temperature_celsius', 'new_deaths_30_lag', 'rainfall_mm', 'new_cases_150_lag', 'new_deaths_7_lag', 'new_deaths_2_lag', 'new_cases_50_lag', 'new_cases_50_mean', 'new_deaths_50_std', 'Series_Complete_5Plus', 'maximum_temperature_celsius']


In [26]:
model_g = xgb.XGBRegressor().fit(X_train, y_train, verbose=False)
predictors_6 = pd.concat([pd.Series(model_g.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_6[1].to_list())

['new_cases_10_max', 'Series_Complete_65Plus', 'new_cases_10_mean', 'new_hospitalized_patients', 'Administered_Dose1_Recip_65Plus', 'new_cases_7_lag', 'new_cases_10_min', 'Series_Complete_Yes', 'new_deaths_10_min', 'new_cases_1 lag', 'new_cases_100_max', 'new_cases_100_mean', 'current_intensive_care_patients', 'new_cases_50_max', 'new_cases_4_lag', 'new_deaths_10_max', 'new_deaths_100_max', 'new_deaths_4_lag', 'new_cases_50_min', 'new_deaths_50_std', 'new_cases_3_lag', 'new_cases_10_std', 'new_cases_50_std', 'school_closing', 'Series_Complete_5Plus', 'new_deaths_50_mean', 'new_cases_100_std', 'new_cases_30_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_150_lag']


In [27]:
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss', allow_missing=True) # note, terms in brackets are the hyperparameters 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
----------------------------------------------------------------------------
Basis Function                                        Pruned  Coefficient   
----------------------------------------------------------------------------
(Intercept)                                           No      -4.96435e+06  
h(new_cases_10_mean-9356.9)                           Yes     None          
h(9356.9-new_cases_10_mean)                           No      -1.8297       
present(new_deaths_4_lag)                             Yes     None          
missing(new_deaths_4_lag)                             No      2383.01       
new_deaths_4_lag*present(new_deaths_4_lag)            No      -1.83932      
h(booster_rate-0.115872)                              No      -3.85057e+08  
h(0.115872-booster_rate)                              No      3.84561e+08   
present(new_cases_4_lag)                              Yes     None          
missing(new_cases_4_lag)                              Yes     No

In [30]:
def common_member(a, b, c):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
 
    if (set_1 & set_2 & set_3):
        print(set_1 & set_2 & set_3)


In [31]:
a = predictors_7[1].to_list()
b = predictors_8[1].to_list()
c = predictors_6[1].to_list()

In [32]:
full = common_member(a, b, c)
full

{'PEOPLE_DEATH_NEW_COUNT', 'Series_Complete_5Plus', 'new_cases_50_std', 'new_cases_4_lag', 'new_cases_10_max', 'new_cases_30_lag', 'new_cases_10_mean', 'new_cases_10_min', 'new_cases_7_lag', 'new_hospitalized_patients', 'new_cases_10_std', 'new_cases_1 lag', 'new_deaths_4_lag', 'new_cases_3_lag'}
